In [18]:
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm

In [19]:
file_path = '../pyanomaly-master/output/merge.pickle'
with open(file_path, 'rb') as f:
    obj = pkl.load(f)
    
df = pd.DataFrame(obj)

In [20]:
# Get indexes
dates = df.index.get_level_values('date')
permno_ids = df.index.get_level_values('permno')
dates[0:10]

DatetimeIndex(['2010-01-31', '2010-02-28', '2010-03-31', '2010-04-30',
               '2010-05-31', '2010-06-30', '2010-07-31', '2010-08-31',
               '2010-09-30', '2010-10-31'],
              dtype='datetime64[ns]', name='date', freq=None)

In [32]:
# Count unique permnos
count = 0
permno_set = set()
for p in permno_ids.to_list():
    if p not in permno_set:
        count += 1
        permno_set.add(p)
count

9206

In [33]:
exclude_columns = ["gvkey", "datadate", "primary", "exchcd", "ret", "exret", "rf", "me"]
exret = df.exret
features = df.drop(columns=exclude_columns)
exret.head()

date        permno
2010-01-31  10001    -0.018960
2010-02-28  10001    -0.000666
2010-03-31  10001     0.020575
2010-04-30  10001     0.124274
2010-05-31  10001     0.004717
Name: exret, dtype: float64

In [34]:
p0 = permno_set.pop()
idxs = [idx for idx, val in enumerate(permno_ids) if val == p]
temp_df = features.iloc[idxs]
temp_exret = exret.iloc[idxs]
i=90
x_temp = temp_df.iloc[i:i+5]
y_temp = temp_exret.iloc[i+5:i+5+1]

In [35]:
x_temp.to_numpy()

array([[ 9.40000000e+01,  1.66704276e-01,  5.34643530e-01, ...,
         2.51832431e-02,  1.34928009e+00,  1.09124034e-01],
       [ 9.50000000e+01,  1.59252012e-01,  4.50969071e-01, ...,
         1.19740330e-02, -5.46508811e-01,  1.09631669e-01],
       [ 9.60000000e+01,  1.59466296e-01,  4.84702059e-01, ...,
         1.19740330e-02, -5.46508811e-01,  1.09631669e-01],
       [ 9.70000000e+01,  1.67951882e-01,  4.59847269e-01, ...,
         1.19740330e-02, -5.46508811e-01,  1.09631669e-01],
       [ 9.80000000e+01,  3.30914054e-01,  3.58832925e-01, ...,
         1.90965471e-02,  8.64726497e-02,  1.21669637e-01]])

In [36]:
x_temp

,,age,mispricing_mgmt,mispricing_perf,qmj,qmj_growth,qmj_prof,qmj_safety,beta_60m,chcsho_12m,chmom,...,niq_at,niq_at_chg1,niq_be,niq_be_chg1,niq_su,ocfq_saleq_std,roavol,rsup,saleq_su,stdacc
date,permno,,,,,,,,,,,,,,,,,,,,,
2017-12-31,93436,94.0,0.166704,0.534644,-0.664615,1.093642,-1.208096,-0.708078,1.199302,0.044788,-0.831205,...,-0.013427,0.018470,-0.067445,0.234697,0.187433,0.152166,0.011885,0.025183,1.349280,0.109124
2018-01-31,93436,95.0,0.159252,0.450969,-1.041790,0.694345,-1.291706,-0.951286,1.226342,0.044084,-0.188624,...,-0.023782,-0.025625,-0.121309,-0.129990,-2.937532,0.136547,0.011907,0.011974,-0.546509,0.109632
2018-02-28,93436,96.0,0.159466,0.484702,-1.047236,0.700880,-1.291295,-0.945867,1.240570,0.044844,-0.459733,...,-0.023782,-0.025625,-0.121309,-0.129990,-2.937532,0.136547,0.011907,0.011974,-0.546509,0.109632
2018-03-31,93436,97.0,0.167952,0.459847,-1.058551,0.698263,-1.286568,-1.004619,1.379444,0.034027,-0.445443,...,-0.023782,-0.025625,-0.121309,-0.129990,-2.937532,0.136547,0.011907,0.011974,-0.546509,0.109632
2018-04-30,93436,98.0,0.330914,0.358833,-0.469646,1.646153,-1.008476,-1.097450,1.342683,0.033690,-0.169096,...,-0.024028,-0.014392,-0.143341,-0.098075,-1.533735,0.130207,0.011355,0.019097,0.086473,0.121670


In [ ]:
# Build data for lstm
past = 5 # 5 months of data
future = 1 # next month risk prem
X_arr,y = [],[]

for p in tqdm(permno_set):
    idxs = [idx for idx, val in enumerate(permno_ids) if val == p]
    temp_df = df.iloc[idxs]
    temp_exret = exret.iloc[idxs]
    
    for i in range(len(temp_df) - past - future + 1):
        X_arr.append(temp_df.iloc[i:i+past])
        y.append(temp_exret.iloc[i+past:i+past+future])
        

 38%|██████████████████████████████████████████████████████▉                                                                                           | 3464/9206 [41:41<15:48,  6.05it/s]